In [237]:
import numpy as np 
import pandas as pd
from scipy.optimize import minimize

In [258]:
#Reading in the Yield Curve Data and Processing it a bit
excel_file = pd.read_excel('yieldcurve2024 (1).xlsx', index_col= [0])
yield_curve = pd.DataFrame(excel_file.iloc[0,:]) / 100
yield_curve['Yield'].to_numpy()[0]

np.float64(0.04479120261500492)

In [269]:
def Building_Tree(theta,
                  number_of_periods = yield_curve['Yield'].to_numpy().shape[0],
                  r0 = r0,
                  volatility = rate_vol,
                  dt = dt):

 
 #Creating Binomial Tree of length i, height i 
 binomial_tree = np.zeros((number_of_periods , number_of_periods))
 binomial_tree[np.triu_indices(number_of_periods, 0)] = r0
 
 #Apply Ho-Lee
 for i in range(1, number_of_periods): #Time steps from i = 1,2, ... N-1
      for j in range(i + 1):      #States from 0 to i 
         
            if j == 0:
                previous_rate = binomial_tree[j, i-1]
                binomial_tree[j , i] = previous_rate + theta[i-1]*dt + volatility*np.sqrt(dt)
            else:
                previous_rate = binomial_tree[j-1, i-1]
                binomial_tree[j, i] = previous_rate + theta[i-1]*dt - volatility*np.sqrt(dt)

 return binomial_tree

In [233]:
def ZCB(yield_curve=yield_curve['Yield'].to_numpy(), dt = dt, periods = 50):
    
    if isinstance(yield_curve, np.ndarray):
        yield_curve = list(yield_curve)
        
    ZCB_prices = np.zeros(periods)
    
    for i in range(periods):
        discount_factor = np.exp( - yield_curve[i] * (dt*(i+1)))
        ZCB_prices[i] = 100.00 * discount_factor
    
    return ZCB_prices


In [234]:
def Ho_Lee_Model_LossFunction(theta, periods, r0 = r0, volatility = rate_vol, dt = dt, 
                              yield_curve = yield_curve['Yield']):
    '''  
    This calculates the error function between the Ho-lee model (using initial theta values that 
    are basically random values) and the actual observed ZCB prices calculated from ZCB() function.

    The idea is to use this function in tandem with an optimiser that will fit the theta values by
    minimising this loss function.

    Not the most computationally efficient method but shows the information flow well.

    Loss function used is MSE.
    Uses the short-rate at time = 0 as an initial value (r0)
    '''
    Error_SSE = 0.
    ZCBx = ZCB(yield_curve=yield_curve, periods = periods)

    for period in range(1, periods + 1):
        target_model_price = ZCB_Ho_Lee_Calibration(maturity = period - 1, theta = theta, r0 = r0)
        observed_price = ZCBx[period - 1]
        Error_SSE += (target_model_price - observed_price)**2

    return Error_SSE


In [272]:
def ZCB_Ho_Lee_Calibration(maturity, theta, r0 = r0, dt = dt, par_value = 100, volatility = rate_vol, test=False):
    ''' 
    To calibrate Ho-Lee model, we use backward induction to calculate the price of ZCB bonds.
    This function will be passed through to an optimiser, we can minimise some loss function by comparing this to the true observed
    ZCB prices. This function manually sets the final layer based on the payoff

    Risk neutral probabilities uses q = 0.5
    Parameters :
        1. maturity : ZCB for a specific maturity 
        2. theta : theta vector
        3. dt : set to 0.5 years
        4. par value : $100 ZCB 
    ----------
    Returns ZCB_price : present value of cash flows e.g., price of ZCB bond at t = 0
    
    '''
    periods = maturity + 1
    #Final node is based on the defined payoff below (for ZCB it's simple)
    binomial_tree = Building_Tree(theta, number_of_periods=periods, dt=dt, volatility=volatility, r0 = r0)
    
    #Create Value Tree and set last row to $100
    value_tree = np.zeros(periods * periods).reshape(periods,periods)

    #Penultimate Node
    value_tree[:,-1] = par_value * np.exp(-binomial_tree[:,-1] * dt)

    #Backward Induction : start from final node, go backwards to i = 0 period
    for i in range(periods-2, -1, -1):

        #Only goes up to when i = j (diagonal)
        for j in range(i+1): 
            #Discounted Expected Value of the bond at time i
            value_tree[j, i] = 0.5 * (value_tree[j, i+1] + value_tree[j+1, i+1]) * np.exp(-binomial_tree[j, i] * dt)
    
    if test == True:
        return value_tree[0,0], value_tree
    else:
        return value_tree[0,0]

In [296]:
def calibration_HL(yield_curve = yield_curve['Yield'].to_numpy(), periods = 50, sigma = rate_vol, dt = dt, r0 = yield_curve['Yield'].iloc[0]):
    ''' 
    argmin theta vector of norm-2 errors 

    '''
    initial_guess_theta = np.zeros(periods-1) * 0.01

    #objective function 
    def objective_function(theta):
        return Ho_Lee_Model_LossFunction(theta=theta, periods = periods, r0 = r0, yield_curve=yield_curve)
    
    result = minimize(objective_function, initial_guess_theta, method = 'BFGS',
                      options={
                          'maxiter': 500,
                          'disp': True
                      })
    print(result.success, result.message)

    theta_calibrated = result.x

    return theta_calibrated, result


In [ ]:
calibration_HL()

### Testing


In [248]:
test_maturity = [i * 0.5 for i in range(1,12)]
test_price = [99.1338, 97.8925, 96.1462, 94.1011, 91.7136, 89.2258, 86.8142, 84.5016, 82.1848, 79.7718, 77.4339]
test_yields = [1.74, 2.13, 2.62, 3.04, 3.46, 3.80, 4.04, 4.21, 4.36, 4.52, 4.65]
test_yields = np.array(test_yields) / 100

In [273]:
pd.DataFrame(Building_Tree(theta = test_thetas, r0 = 0.0174, number_of_periods=11))

,0,1,2,3,4,5,6,7,8,9,10
0,0.0174,0.037470,0.060615,0.080035,0.100930,0.117100,0.129544,0.141463,0.155857,0.172726,0.185559
1,0.0000,0.013005,0.036149,0.055569,0.076464,0.092634,0.105078,0.116997,0.131391,0.148260,0.161094
2,0.0000,0.000000,0.011684,0.031104,0.051999,0.068168,0.080612,0.092531,0.106925,0.123794,0.136628
3,0.0000,0.000000,0.000000,0.006638,0.027533,0.043702,0.056147,0.068065,0.082459,0.099328,0.112162
4,0.0000,0.000000,0.000000,0.000000,0.003067,0.019236,0.031681,0.043600,0.057994,0.074862,0.087696
5,0.0000,0.000000,0.000000,0.000000,0.000000,-0.005230,0.007215,0.019134,0.033528,0.050396,0.063230
6,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.017251,-0.005332,0.009062,0.025930,0.038764
7,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.029798,-0.015404,0.001464,0.014298
8,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.039870,-0.023002,-0.010168
9,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.047468,-0.034634


In [275]:
ZCB_Ho_Lee_Calibration(maturity=10, theta=test_thetas, r0 = 0.0174, test=True)[0]

np.float64(77.43389845384934)

In [287]:
pd.DataFrame(ZCB_Ho_Lee_Calibration(maturity=10, theta=test_thetas, r0 = 0.0174, test=True)[1])

,0,1,2,3,4,5,6,7,8,9,10
0,77.433898,73.338852,70.616448,69.230555,68.973914,69.882931,71.831375,74.763387,78.770778,84.113146,91.139423
1,0.000000,82.882172,78.835201,76.348309,75.140448,75.205104,76.362069,78.512675,81.715274,86.196430,92.261173
2,0.000000,0.000000,88.010501,84.197857,81.858294,80.932606,81.178532,82.449985,84.769837,88.331312,93.396731
3,0.000000,0.000000,0.000000,92.854434,89.176742,87.096306,86.298789,86.584746,87.938580,90.519070,94.546265
4,0.000000,0.000000,0.000000,0.000000,97.149487,93.729423,91.742001,90.926860,91.225774,92.761014,95.709948
5,0.000000,0.000000,0.000000,0.000000,0.000000,100.867708,97.528538,95.486725,94.635844,95.058485,96.887953
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,103.680056,100.275262,98.173385,97.412860,98.080457
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,105.303938,101.843161,99.825547,99.287639
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,105.650115,102.297990,100.509679
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,104.831670,101.746759


In [295]:
Ho_Lee_Model_LossFunction(theta = x, periods = 11, r0 = 0.0174, yield_curve=test_yields)

np.float64(2.7772040596424528e-11)

In [293]:
x,y = calibration_HL(yield_curve=test_yields, periods = 11, r0 = 0.0174)
x

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 17
         Function evaluations: 308
         Gradient evaluations: 28
True Optimization terminated successfully.


array([ 0.01567471,  0.02182459,  0.01437408,  0.01732368,  0.00787323,
        0.00042274, -0.00062782,  0.00432155,  0.00927083,  0.00122003])

In [262]:
test_thetas

array([ 0.015675,  0.021824,  0.014374,  0.017324,  0.007873,  0.000423,
       -0.000628,  0.004322,  0.009271,  0.001202])